In [65]:
import cv2
from matplotlib import pyplot as plt
import math
import os
import pandas as pd
import pickle
import shutil
import numpy as np

In [66]:
def display_image(img, title="image"):
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

In [91]:
def calculate_huMoments(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    moments = cv2.moments(binary)
    hu_moments = cv2.HuMoments(moments).flatten()
    #Log transform the HuMoments
    for i in range(0,7):
        if hu_moments[i] == 0:
            hu_moments[i] = 0
        else:
            hu_moments[i] = -1* math.copysign(1.0, hu_moments[i]) * math.log10(abs(hu_moments[i]))
    
    hu_moments = np.abs(hu_moments)
    
    return hu_moments


In [68]:
tsv_file = "rumi-jawi.tsv"
mapping_df = pd.read_csv(tsv_file, sep="\t")

rumi_to_jawi = dict(zip(mapping_df['rumi'], mapping_df['jawi']))

In [92]:
#Getting HuMoments of all images in the samplewords folder.

folder_path = "samplewords"

missing_folder = "missingfiles"
os.makedirs(missing_folder, exist_ok=True)

missing_entries = []
jawi_huMoments = {}

for image in os.listdir(folder_path):
    if image.endswith(".png"):
        rumi_name = os.path.splitext(image)[0]
        jawi_name = rumi_to_jawi.get(rumi_name)

        if not jawi_name:
            print(f"Skipping {image}: No corresponding Jawi name found in .tsv.")
            shutil.move(os.path.join(folder_path, image), os.path.join(missing_folder, image))
            continue

        image_path = os.path.join(folder_path, image)
        try:
            hu_moments = calculate_huMoments(image_path)
        except Exception as e:
            print(f"Skipping {image}: Error computing Hu moments for {rumi_name}.")
            continue

        jawi_huMoments[jawi_name] = hu_moments

for rumi_name in rumi_to_jawi.keys():
    jawi_name = rumi_to_jawi.get(rumi_name)
    image = os.path.join(folder_path, f"{rumi_name}.png")
    if not os.path.exists(image):
        print(f"Warning: Image file not found for {rumi_name}.")
        missing_entries.append({'rumi': rumi_name, 'jawi': jawi_name})

missing_entries_df = pd.DataFrame(missing_entries)
missing_entries_tsv = os.path.join(missing_folder, 'missing_entries.tsv')
missing_entries_df.to_csv(missing_entries_tsv, sep='\t', index=False)
    
with open('hu_moments_dict.pkl', 'wb') as f:
    pickle.dump(jawi_huMoments, f)

In [69]:
len(jawi_huMoments)

9517

In [70]:
def compare_hu_moments(hu1, hu2):
    return np.sqrt(np.sum((hu1-hu2) ** 2))

def find_closest_match(new_hu, hu_moments_dict):
    closest_word = None
    min_distance = float('inf')

    for word, hu in hu_moments_dict.items():
        distance = compare_hu_moments(new_hu, hu)
        if distance < min_distance:
            min_distance = distance
            closest_word = word

    return closest_word, min_distance

In [96]:
new_image = "sample3.jpg"
new_hu = calculate_huMoments(new_image)

closest_word, min_distance = find_closest_match(new_hu, jawi_huMoments)
print(f"The closest match is: {closest_word} with a distance of {min_distance}")

The closest match is: دوݢاءن with a distance of 0.23036958444973862


In [75]:
# Comparing the actual huMoment of the word vs the sample image vs the closest word predicted.

print(jawi_huMoments["اچارا"])
print(calculate_huMoments(new_image))
print(jawi_huMoments["کوادرن"])

[ 3.08327132  6.82830533 12.53898352 12.22653836 24.92605759 15.89808256
 24.66676928]
[  3.0938976    6.91520789  13.21372614  12.82217799  25.92574821
  16.87348934 -26.0836286 ]
[  3.01512602   6.41619617  13.25446531  12.57116662  26.03585272
  17.20883176 -25.50179423]


In [93]:
new_image = "sample2.jpg"

hu_moments1 = calculate_huMoments(new_image)
hu_moments2 = jawi_huMoments["اچارا"]
hu_moments3 = jawi_huMoments["ڤلاريق"]

# Determining the Euclidean distance of the Hu Moments.
distance = np.linalg.norm(hu_moments1 - hu_moments2)
print(f"Euclidean Distance: {distance}.")

Euclidean Distance: 1.4187780303911006.


In [19]:
# Trying to perform some image pre-processing.

img = cv2.imread("sample2.jpg", cv2.IMREAD_GRAYSCALE)
_, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
moments = cv2.moments(binary)
hu_moments = cv2.HuMoments(moments).flatten()

for i in range(0, 7):
    if hu_moments[i] == 0:
        hu_moments[i] == 0
    else:
        hu_moments[i] = -1* math.copysign(1.0, hu_moments[i]) * math.log10(abs(hu_moments[i]))

hu_moments


array([  3.0938976 ,   6.91520789,  13.21372614,  12.82217799,
        25.92574821,  16.87348934, -26.0836286 ])

---

In [45]:
# Trying different features.

from skimage.feature import hog, local_binary_pattern
import mahotas

def calculate_hog_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (64, 128))
    hog_features, _ = hog(image, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), visualize=True, block_norm='L2-Hys')

    return hog_features

def calculate_sift_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(image, None)

    return descriptors

def calculate_lbp_features(image_path, radius=1, n_points=8):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    lbp = local_binary_pattern(image, n_points, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points+3), range=(0, n_points+2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)

    return hist

def calculate_zernike_moments(image_path, radius=21):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    zernike_moments = mahotas.features.zernike_moments(binary, radius)

    return zernike_moments

def calculate_gabor_features(image_path, ksize=31, sigma=4.0, theta=np.pi/4, lambd=10.0, gamma=0.5, psi=0):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
    filtered_image = cv2.filter2D(image, cv2.CV_8UC3, gabor_kernel)
    hist = cv2.calcHist([filtered_image], [0], None, [256], [0,256])

    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)

    return hist

In [47]:
# Trying to see what the other features may look like.

image_path = "sample3.jpg"
hog_features = calculate_hog_features(image_path)
sift_features = calculate_sift_features(image_path)
lbp_features = calculate_lbp_features(image_path)
zernike_features = calculate_zernike_moments(image_path)
gabor_features = calculate_gabor_features(image_path)

print("HOG Features: ", hog_features)
print("SIFT Features: ", sift_features)
print("LBP Features: ", lbp_features)
print("Zernike Moments: ", zernike_features)
print("Gabor Features: ", gabor_features)

HOG Features:  [0. 0. 0. ... 0. 0. 0.]
SIFT Features:  [[  0.   0.  71. ...   0.   0.   1.]
 [  0.   4.  46. ...   0.   0.   6.]
 [ 14.  37. 157. ...   0.   0.   2.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  6.   0.   0. ...  18.  12.   0.]
 [  0.   0.   0. ...   0.   1.   6.]]
LBP Features:  [0.00730857 0.01901269 0.0020027  0.02156159 0.02135352 0.05547753
 0.00382335 0.02320017 0.79928735 0.04697253]
Zernike Moments:  [0.31830989 0.03962973 0.05296362 0.0572742  0.0411407  0.05364851
 0.06392936 0.06299502 0.03106492 0.04008593 0.01432621 0.02601447
 0.06891804 0.06496817 0.03611487 0.01110981 0.01695589 0.01148972
 0.03917521 0.00357495 0.01473234 0.04673428 0.06783395 0.01494838
 0.01115743]
Gabor Features:  [[3.57105701e-02]
 [2.60091552e-05]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [5.20183104e-05]
 [0.00000000e+00]
 [2.60091552e-05]
 [5.20183104e-05]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2

In [57]:
def combine_features(hog_features, hu_moments):
    return np.concatenate((hog_features, hu_moments))

In [58]:
from scipy.spatial.distance import euclidean, cosine

def compare_images(image1, image2):
    #Extract HOG features
    hog_feat1 = calculate_hog_features(image1)
    hog_feat2 = calculate_hog_features(image2)

    #Extract Hu Moments
    hu_mom1 = calculate_huMoments(image1)
    hu_mom2 = calculate_huMoments(image2)

    #Combine features
    combined_features1 = combine_features(hog_feat1, hu_mom1)
    combined_features2 = combine_features(hog_feat2, hu_mom2)

    euclidean_distance = euclidean(combined_features1, combined_features2)
    cosine_similarity = 1 - cosine(combined_features1, combined_features2)

    return euclidean_distance, cosine_similarity

In [61]:
image1 = "sample3.jpg"
image2 = "samplewords/agregat.png"
image3 = "samplewords/zoofit.png"

euclidean_distance, cosine_similarity = compare_images(image1, image2)

print(f"Euclidean Distance between combined features: {euclidean_distance}.")
print(f"Cosine Similarity between combined features: {cosine_similarity}.")

# display_image(image1, "Image 1")
# display_image(image2, "Image 2")

Euclidean Distance between combined features: 9.706802546086799.
Cosine Similarity between combined features: 0.976940946177653.


---

In [55]:
#Trying to work with HuMoments once more.

import cv2

def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    reverse = cv2.bitwise_not(binary)

    return reverse

In [23]:
def extract_hu_moments(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    hu_moments_list = []
    for contour in contours:
        moments = cv2.moments(contour)
        hu_moments = cv2.HuMoments(moments).flatten()
        hu_moments_list.append(hu_moments)

    log_humoments = []
    for hu_moments in hu_moments_list:
        log_humoments_value = np.sign(hu_moments) * np.log10(np.abs(hu_moments) + np.finfo(float).eps)
        log_humoments.append(log_humoments_value)

    return log_humoments

In [47]:
import numpy as np

def calculate_hu_distance(hu_moments1, hu_moments2):
    distances = []
    for moments1 in hu_moments1:
        for moments2 in hu_moments2:
            distance = np.linalg.norm(moments1 - moments2)
            distances.append(distance)

    return min(distances)

In [57]:
import os

sample = "sample2.jpg" #should be "acara.png"
folder_path = "samplewords"

min_distance = float('inf')
most_similar_word = None

samp_binary = preprocess_image(sample)
samp_huMom = extract_hu_moments(samp_binary)

for image in os.listdir(folder_path):
    image_file = os.path.join(folder_path, image)

    new_bin = preprocess_image(image_file)
    new_huMom = extract_hu_moments(new_bin)
    distance = calculate_hu_distance(samp_huMom, new_huMom)
    if distance < min_distance:
        min_distance = distance
        most_similar_word = image

most_similar_word

'lencana.png'

In [46]:
sample = "sample2.jpg"
sim = "sample.jpg"
notsim = "sample3.jpg"
acara = "samplewords/acara.png"

psam = preprocess_image(sample)
paca = preprocess_image(acara)
pnsim = preprocess_image(notsim)

hsam = extract_hu_moments(psam)
haca = extract_hu_moments(paca)
hnsim = extract_hu_moments(pnsim)

calculate_hu_distance(hsam, haca)

0.08498511122737039